In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dot, Activation, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
num_classes = len(chars)

char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_char = {idx: char for idx, char in enumerate(chars)}

In [3]:
# 데이터 생성 함수
def generate_data(num_samples):
    X = []
    y = []
    for _ in range(num_samples):
        sequence = np.random.choice(list(chars), 15)
        X.append([char_to_index[char] for char in sequence])
        y.append([char_to_index[char] for char in sequence[::-1]])
    return np.array(X), np.array(y)

In [4]:
# 데이터 생성
num_samples = 20000
X, y = generate_data(num_samples)

In [5]:
#인코더 정의
encoder_inputs = Input(shape = (15,))
encoder_embedding = Embedding(input_dim = num_classes, output_dim = 64, input_length = 15)(encoder_inputs)
encoder = LSTM(128, return_sequences = True, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_embedding)
encoder_states = [state_h, state_c]

In [6]:
# 디코더 정의
decoder_inputs = Input(shape = (15, ))
decoder_embedding = Embedding(input_dim = num_classes, output_dim = 64, input_length = 15)(decoder_inputs)
decoder_lstm = LSTM(128, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state = encoder_states)

In [7]:
# 어텐션 메커니즘
attention = Dot(axes = [2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)
context = Dot(axes = [2, 1])([attention, encoder_outputs])

In [8]:
# 디코더 출력과 컨텐스트 결합
decoder_combined_context = Concatenate(axis = -1)([decoder_outputs, context])

In [9]:
# 출력 레이어
decoder_dense = Dense(num_classes, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_combined_context)

In [10]:
# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [11]:
# 입력 데이터와 타겟 데이터를 동일하게 맞춤
decoder_input_data = np.zeros_like(X)

In [12]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights=True)

In [13]:
# 모델 학습
model.fit([X,decoder_input_data], y, epochs = 50, validation_split = 0.2, callbacks = [early_stopping])

Epoch 1/50
500/500 [==============================] - 11s 16ms/step - loss: 2.9161 - accuracy: 0.2621 - val_loss: 0.2181 - val_accuracy: 0.9975
Epoch 2/50
500/500 [==============================] - 6s 13ms/step - loss: 0.0370 - accuracy: 0.9998 - val_loss: 0.0092 - val_accuracy: 0.9999
Epoch 3/50
500/500 [==============================] - 6s 13ms/step - loss: 0.0723 - accuracy: 0.9903 - val_loss: 0.0054 - val_accuracy: 0.9999
Epoch 4/50
500/500 [==============================] - 9s 18ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 5/50
500/500 [==============================] - 8s 16ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 6/50
500/500 [==============================] - 10s 20ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 0.9999
Epoch 7/50
500/500 [==============================] - 10s 20ms/step - loss: 7.2511e-04 - accuracy: 1.0000 - val_loss: 7.9214e-04 - val_accur

In [14]:
# 모델 검증 함수
def predict_reverse(input_sequence):
    input_sequence = np.array([char_to_index[char] for char in input_sequence]).reshape(1, 15)
    decoder_input = np.zeros((1, 15))
    predicted_sequence = model.predict([input_sequence, decoder_input])
    predicted_indices = np.argmax(predicted_sequence, axis = -1).reshape((15, ))
    return ''.join([index_to_char[idx] for idx in predicted_indices])

In [15]:
# 검증용 문자열 리스트
test_strings = ["".join(np.random.choice(list(chars), 15)) for _ in range(30)]

In [16]:
# 실제 거꾸로 된 문자열 리스트
expected_outputs = [s[::-1] for s in test_strings]

In [18]:
# 모델 검증 및 정확도 계산
correct_predictoins = 0
total_predictions = len(test_strings)

for i, test_string in enumerate(test_strings):
    predicted_output = predict_reverse(test_string)
    is_correct = predicted_output == expected_outputs[i]

    if is_correct:
        correct_predictoins += 1
        print(f"입력: {test_string}")
        print(f"예측된 출력: {predicted_output}")
        print(f"실제 출력: {expected_outputs[i]}")
        print(f"정확 여부 {'맞음' if is_correct else '틀림'}\n")

accuracy = correct_predictoins / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")

1/1 [==============================] - 0s 49ms/step
입력: dDaEuwjXZcjPxpM
예측된 출력: MpxPjcZXjwuEaDd
실제 출력: MpxPjcZXjwuEaDd
정확 여부 맞음

1/1 [==============================] - 0s 44ms/step
입력: OjatQODlZsEspQv
예측된 출력: vQpsEsZlDOQtajO
실제 출력: vQpsEsZlDOQtajO
정확 여부 맞음

1/1 [==============================] - 0s 41ms/step
입력: NGTECRrucEoWUAq
예측된 출력: qAUWoEcurRCETGN
실제 출력: qAUWoEcurRCETGN
정확 여부 맞음

1/1 [==============================] - 0s 45ms/step
입력: KjNjGVIjJKnpDfr
예측된 출력: rfDpnKJjIVGjNjK
실제 출력: rfDpnKJjIVGjNjK
정확 여부 맞음

1/1 [==============================] - 0s 46ms/step
입력: avZpMlIBblqBWtb
예측된 출력: btWBqlbBIlMpZva
실제 출력: btWBqlbBIlMpZva
정확 여부 맞음

1/1 [==============================] - 0s 42ms/step
입력: DkPZbWeswnOfyHp
예측된 출력: pHyfOnwseWbZPkD
실제 출력: pHyfOnwseWbZPkD
정확 여부 맞음

1/1 [==============================] - 0s 50ms/step
입력: hqxCtRjKCZARsEl
예측된 출력: lEsRAZCKjRtCxqh
실제 출력: lEsRAZCKjRtCxqh
정확 여부 맞음

1/1 [==============================] - 0s 43ms/step
입력: CqAMNlmNHDwIbCR
예측된 출력: RCbIwDHNmlNMAqC
실